In [1]:
from maelzel.core import *

In [2]:
presetManager.showPresets('*', full=True)


In [3]:
presetManager

Preset: sqr  (square wave with optional filtering)
    hasRouting=True
  |ktransp=0.0, klag=0.1, kcutoff=0.0, kresonance=0.2|
  ; Square wave with optional filtering
  ; Args:
  ;    kcutoff: filter cutoff frequency
  ;    kresonance: resonance of the filter
  aout1 = vco2(1, mtof(lag(kpitch+ktransp, klag), 10) * a(kamp)
  aout1 = kcutoff == 0 ? aout1 : moogladder aout1, lag(kcutoff, 0.1), kresonance

In [2]:
e = playEngine(latency=0)
s = e.session()



--Csound version 6.18 (double samples) Oct  1 2022
[commit: ac38f27d1c5d6ee0cacc1bc719e96be3a2d93a15]
libsndfile-1.0.31
sr = 44100.0, kr = 689.062, ksmps = 64
0dBFS level = 1.0, A4 tuning = 442.0
audio buffered in 256 sample-frame blocks
reading 1024-byte blks of shorts from adc:system:capture (RAW)
writing 512 sample blks of 64-bit floats to dac:system:playback
SECTION 1:


In [9]:
foo = defPreset('foo', r'''
|ibusL, ibusR|
prints "ibusL: %d, ibusR: %d\n", ibusL, ibusR
aout1 vco2 kamp, kfreq
aout2 vco2 kamp, kfreq+4

busout ibusL, aout1
busout ibusR, aout2
''', routing=False)

In [10]:
busL = s.assignBus()
busR = s.assignBus()

s.defInstr('master', r'''
    |ibusL, ibusR|
    aL busin ibusL
    aR busin ibusR
    outch 1, aL, 2, aR
    ''')

Instr(master, ibusL:5=0, ibusR:6=0)

In [11]:
master = s.sched('master', args=dict(ibusL=busL, ibusR=busR), priority=2)

In [15]:
Note(60.5, 10).play(instr='foo', gain=0.1, ibusL=busL, ibusR=busR)

ibusL: 0, ibusR: 1


Button(description='Stop', style=ButtonStyle())

Output()

SynthGroup(n=1)
    Synth(▶ preset.foo:502.0005, p4=0 idataidx_:5=17 inumbps:6=2 ibplen:7=3 igain_:8=0.1 ichan:9=1 ipos:10=-1 ifadein:11=0.02 ifadeout:12=0.02 ipchintrp_:13=0 ifadekind_:14=1 ibusL:15=0 ibusR:16=1 ...)

In [7]:
master.stop()

In [3]:
foo = s.defInstr('foo', r'''
    |itime, Sfoo|
    ielapsed elapsedtime
    prints "Sfoo: %s, itime: %f, ielapsed: %f, diff: %f\n", Sfoo, itime, ielapsed, ielapsed - itime 
    turnoff
''', priority=1)

In [8]:
s.sched("foo", 0, 1, args={'Sfoo': 'bar', 'itime': e.elapsedTime()})

Sfoo: bar, itime: 18.031746, ielapsed: 18.030295, diff: -0.001451


Button(description='Stop', style=ButtonStyle())

Output()

Synth(▶ foo:501.0006, p4=0 itime:5=18.0317 Sfoo:6=bar)

In [19]:
e.extraLatency

0.2

In [5]:
s.engine._strToIndex

{'cos': 20, 'linear': 21, 'smooth': 22, 'smoother': 23, 'hola': 24}

In [2]:
test = s.defInstr('test', r'''
    |istrid|
    S0 = strget(istrid)
    prints "S0: %s\n", S0
    turnoff
''')

In [3]:
jiji = s.engine.strSet('jiji')

In [4]:
s.sched('test', 0, 1, pargs={'istrid': jiji})

S0: jiji
--- dealloc 501.000200


Synth(■ test:501.0002, p4=0 istrid:5=24)

In [2]:
e = playEngine()
s = e.session()



--Csound version 6.18 (double samples) Aug 23 2022
[commit: a691b9fb3a19ff27df8df4cb44b9c6be0d558fbc]
libsndfile-1.0.31
sr = 44100.0, kr = 689.062, ksmps = 64
0dBFS level = 1.0, A4 tuning = 442.0
audio buffered in 256 sample-frame blocks
reading 1024-byte blks of shorts from adc:system:capture (RAW)
writing 512 sample blks of 64-bit floats to dac:system:playback
SECTION 1:


In [3]:
test = s.defInstr('test', r'''
    |ichan=0|
    prints "ichan: %d\n", ichan
    Schan = "bar"
    ; Schan strget ichan
    prints "Schan: %s\n", Schan
    a0 oscili 0.1, 1000
    ; chnset a0, Schan
''')


In [4]:
print(test.body)

ichan = p5
prints "ichan: %d\n", ichan
Schan = "bar"
; Schan strget ichan
prints "Schan: %s\n", Schan
a0 oscili 0.1, 1000
; chnset a0, Schan


In [16]:
ichan = s.engine.strSet('foo')
ichan

24

In [ ]:
s.sched('test', pargs={'ichan': 0})

In [3]:
e.testAudio()

24.0002


Testaudio: pink noise mode
Channel: 1 / 2
Channel: 2 / 2
Channel: 1 / 2
Channel: 2 / 2


In [6]:
bus = e.assignBus()

In [7]:
bus

1

In [2]:
e = playEngine()
bus = e.assignBus()
bus



--Csound version 6.18 (double samples) Aug 23 2022
[commit: a691b9fb3a19ff27df8df4cb44b9c6be0d558fbc]
libsndfile-1.0.31
sr = 44100.0, kr = 689.062, ksmps = 64
0dBFS level = 1.0, A4 tuning = 442.0
audio buffered in 256 sample-frame blocks
reading 1024-byte blks of shorts from adc:system:capture (RAW)
writing 512 sample blks of 64-bit floats to dac:system:playback
SECTION 1:


0

In [3]:
foo = defPreset('foo', audiogen=r'''
    |ibus=0|
    a1 = oscili(kamp*0.5, kfreq)
    a2 = oscili(kamp*0.5, kfreq+12)
    aout1 = a1+a2
''', out=r'''
    replaceout ichan, aout1
    busout ibus, aout1
    busout 
''')
foo

Preset: foo  
    hasRouting=False
  |ibus=0.0|
  a1 = oscili(kamp*0.5, kfreq)
  a2 = oscili(kamp*0.5, kfreq+12)
  aout = a1+a2
  if ibus >= 0 then
      busout ibus, aout
  else
      outch 1, aout
  endif

In [13]:
print(foo.body)


;5          6       7      8     9     0    1      2      3          4        
idataidx_,inumbps,ibplen,igain_,ichan_,ipos,ifade0,ifade1,ipchintrp_,ifadekind_ passign 5
idatalen_ = inumbps * ibplen
iArgs[] passign idataidx_, idataidx_ + idatalen_
ilastidx = idatalen_ - 1
iTimes[]     slicearray iArgs, 0, ilastidx, ibplen
iPitches[]   slicearray iArgs, 1, ilastidx, ibplen
iAmps[]      slicearray iArgs, 2, ilastidx, ibplen

k_time timeinsts

if ipchintrp_ == 0 then      
    ; linear midi interpolation    
    kpitch, kamp bpf k_time, iTimes, iPitches, iAmps
    kfreq mtof kpitch
elseif (ipchintrp_ == 1) then  ; cos midi interpolation
    kidx bisect k_time, iTimes
    kpitch interp1d kidx, iPitches, "cos"
    kamp interp1d kidx, iAmps, "cos"
    kfreq mtof kpitch
elseif (ipchintrp_ == 2) then  ; linear freq interpolation
    iFreqs[] mtof iPitches
    kfreq, kamp bpf k_time, iTimes, iFreqs, iAmps
    kpitch ftom kfreq
elseif (ipchintrp_ == 3) then  ; cos freq interpolation
    kidx bis

In [10]:
Note(60, 1000).play(instr='foo', fade=0.05, params={'ibus': bus})

Button(description='Stop', style=ButtonStyle())

Output()

SynthGroup(n=1)
    Synth(𝍪 preset.foo:501.0006, p4=0 idataidx_:5=16 inumbps:6=2 ibplen:7=3 igain_:8=1 ichan_:9=1 ipos:10=-1 ifade0:11=0.05 ifade1:12=0.05 ipchintrp_:13=0 ifadekind_:14=1 ibus:15=0 p16=0 ...)

In [7]:
s = e.session()
s.defInstr('master', r'''
    |ibus|
    a0 = busin(ibus)
    outch 1, a0
''')

Instr(master, ibus:5=0)

In [8]:
s.sched('master', priority=2)

Synth(𝍪 master:1001.0002, p4=0 ibus:5=0)

In [12]:
post

Button(description='Stop', style=ButtonStyle())

Output()

Synth(▶ post:2001.0002, p4=0 ibus:5=0)

In [15]:
Note(72, dur=10).play(instr='foo', params=dict(ibus=busnum))


Button(description='Stop', style=ButtonStyle())

Output()

SynthGroup(n=1)
    Synth(𝍪 preset.foo:1001.0002, p4=0 idataidx_:5=16 inumbps:6=2 ibplen:7=3 igain_:8=1 ichan_:9=1 ipos:10=-1 ifade0:11=0.02 ifade1:12=0.02 ipchintrp_:13=0 ifadekind_:14=1 ibus:15=0 p16=0 ...)

Csound tidy up: Segmentation fault
